In [64]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# ✅ 1️⃣ 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# ✅ 2️⃣ KREAM 로그인 페이지 접속
driver.get("https://kream.co.kr/login")
wait = WebDriverWait(driver, 10)  # 최대 10초 대기

# ✅ 3️⃣ CSS 선택자로 이메일 & 비밀번호 입력
try:
    email_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#wrap > div.login.layout__main--without-search > div > form > div:nth-child(2) > div > input")))
    password_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#wrap > div.login.layout__main--without-search > div > form > div:nth-child(3) > div > input")))

    email_input.send_keys(USERNAME)  # 이메일 입력
    password_input.send_keys(PASSWORD)  # 비밀번호 입력
    password_input.send_keys(Keys.RETURN)  # 엔터 키로 로그인
    print("✅ 로그인 시도 중...")

    time.sleep(5)  # 로그인 후 페이지 로딩 대기
except Exception as e:
    print("❌ 로그인 필드를 찾을 수 없음:", e)
    driver.quit()
    exit()


✅ 로그인 시도 중...


In [65]:
# ✅ 4️⃣ 로그인 성공 후 특정 페이지로 이동
driver.get("https://kream.co.kr/products/21935")
time.sleep(5)

# ✅ 5️⃣ `#panel1 > a` 버튼 클릭
try:
    panel_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#panel1 > a")))
    panel_button.click()
    print("✅ 버튼 클릭 완료!")
    time.sleep(3)  # 클릭 후 페이지 로딩 대기
except Exception as e:
    print("❌ 버튼을 클릭할 수 없음:", e)
    driver.quit()
    exit()

✅ 버튼 클릭 완료!


In [67]:
# 6️⃣ 스크롤을 내리면서 price_body 내부의 모든 body_list 크롤링
all_data = []  # 데이터를 저장할 리스트
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    try:
        # price_body 내부의 모든 body_list 찾기
        price_body = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".price_body")))
        body_lists = price_body.find_elements(By.CSS_SELECTOR, ".body_list")
        
        for body in body_lists:
            try:
                list_txts = body.find_elements(By.CLASS_NAME, "list_txt")
                if len(list_txts) >= 3:
                    size = list_txts[0].find_element(By.TAG_NAME, "span").text.strip()
                    price = list_txts[1].find_element(By.TAG_NAME, "span").text.strip()
                    # 배송 날짜 추출: 해당 요소 내의 span 중 마지막 요소 사용
                    date_spans = list_txts[2].find_elements(By.TAG_NAME, "span")
                    date = date_spans[-1].text.strip() if date_spans else ""
                    
                    # "빠른배송" 여부 확인: express_icon이 있으면 빠른배송, 없으면 일반배송
                    fast_delivery = "빠른배송" if body.find_elements(By.CLASS_NAME, "express_icon") else "일반배송"
                    
                    all_data.append((size, price, date, fast_delivery))
            except Exception as e:
                print("⚠️ 일부 데이터를 가져오는 데 실패:", e)
    except Exception as e:
        print("❌ 데이터를 찾을 수 없음:", e)
    
    # 스크롤 다운 및 대기시간 5초 설정
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    
    # 마지막 요소가 보이도록 스크롤
    try:
        last_element = price_body.find_elements(By.CSS_SELECTOR, ".body_list")[-1]
        driver.execute_script("arguments[0].scrollIntoView();", last_element)
    except Exception as e:
        print("⚠️ 마지막 요소 스크롤 실패:", e)
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 7️⃣ 크롤링한 데이터 출력
for data in all_data:
    print(f"✅ 사이즈: {data[0]} | 가격: {data[1]} | 배송 날짜: {data[2]} | 배송 방식: {data[3]}")

# 8️⃣ 드라이버 종료
# driver.quit()

✅ 사이즈: 275 | 가격: 146,000원 | 배송 날짜: 25/02/13 | 배송 방식: 일반배송
✅ 사이즈: 265 | 가격: 148,000원 | 배송 날짜: 25/02/13 | 배송 방식: 빠른배송
✅ 사이즈: 250 | 가격: 140,000원 | 배송 날짜: 25/02/13 | 배송 방식: 일반배송
✅ 사이즈: 265 | 가격: 138,000원 | 배송 날짜: 25/02/13 | 배송 방식: 일반배송
✅ 사이즈: 255 | 가격: 179,000원 | 배송 날짜: 25/02/13 | 배송 방식: 빠른배송
✅ 사이즈: 250 | 가격: 140,000원 | 배송 날짜: 25/02/13 | 배송 방식: 일반배송
✅ 사이즈: 260 | 가격: 147,000원 | 배송 날짜: 25/02/13 | 배송 방식: 빠른배송
✅ 사이즈: 300 | 가격: 159,000원 | 배송 날짜: 25/02/13 | 배송 방식: 일반배송
✅ 사이즈: 270 | 가격: 152,000원 | 배송 날짜: 25/02/13 | 배송 방식: 빠른배송
✅ 사이즈: 270 | 가격: 152,000원 | 배송 날짜: 25/02/13 | 배송 방식: 빠른배송
✅ 사이즈: 270 | 가격: 143,000원 | 배송 날짜: 25/02/13 | 배송 방식: 일반배송
✅ 사이즈: 275 | 가격: 146,000원 | 배송 날짜: 25/02/13 | 배송 방식: 일반배송
✅ 사이즈: 245 | 가격: 149,000원 | 배송 날짜: 25/02/13 | 배송 방식: 빠른배송
✅ 사이즈: 265 | 가격: 149,000원 | 배송 날짜: 25/02/13 | 배송 방식: 빠른배송
✅ 사이즈: 270 | 가격: 153,000원 | 배송 날짜: 25/02/13 | 배송 방식: 빠른배송
✅ 사이즈: 260 | 가격: 148,000원 | 배송 날짜: 25/02/13 | 배송 방식: 빠른배송
✅ 사이즈: 275 | 가격: 159,000원 | 배송 날짜: 25/02/13 | 배송 방식: 빠른배송
✅ 사이즈: 260 | 가

In [56]:
all_data

[('275', '146,000원', '25/02/13', '일반배송'),
 ('265', '138,000원', '25/02/13', '일반배송'),
 ('255', '179,000원', '25/02/13', '빠른배송'),
 ('250', '140,000원', '25/02/13', '일반배송'),
 ('260', '147,000원', '25/02/13', '빠른배송'),
 ('300', '159,000원', '25/02/13', '일반배송'),
 ('270', '152,000원', '25/02/13', '빠른배송'),
 ('270', '152,000원', '25/02/13', '빠른배송'),
 ('270', '143,000원', '25/02/13', '일반배송'),
 ('275', '146,000원', '25/02/13', '일반배송'),
 ('245', '149,000원', '25/02/13', '빠른배송'),
 ('270', '153,000원', '25/02/13', '빠른배송'),
 ('260', '148,000원', '25/02/13', '빠른배송'),
 ('265', '149,000원', '25/02/13', '빠른배송'),
 ('275', '159,000원', '25/02/13', '빠른배송'),
 ('260', '148,000원', '25/02/13', '빠른배송'),
 ('265', '149,000원', '25/02/13', '빠른배송'),
 ('260', '147,000원', '25/02/13', '빠른배송'),
 ('265', '138,000원', '25/02/13', '일반배송'),
 ('265', '149,000원', '25/02/13', '빠른배송'),
 ('270', '143,000원', '25/02/13', '일반배송'),
 ('280', '170,000원', '25/02/13', '일반배송'),
 ('285', '205,000원', '25/02/13', '빠른배송'),
 ('270', '145,000원', '25/02/13', '